# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Holualoa,19.6228,-155.9522,301.23,69,0,3.60,US,1697143034
1,1,Ad Dabbah,18.0500,30.9500,305.60,18,0,9.32,SD,1697143027
2,2,Port-Aux-Francais,-49.3500,70.2167,275.34,81,79,7.35,TF,1697142925
3,3,Waitangi,-43.9535,-176.5597,283.61,76,88,4.02,NZ,1697142472
4,4,Port-Cartier,50.0334,-66.8654,284.12,93,100,2.19,CA,1697143034


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
BestWeather = city_data_df.loc[(city_data_df['Max Temp'] >= 70) & (city_data_df['Cloudiness'] <= 50) & 
                                  (city_data_df['Wind Speed'] <= 20) & (city_data_df['Humidity'] <= 20)]

# Drop any rows with null values
BestWeather = BestWeather.dropna()

# Display sample data
BestWeather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Ad Dabbah,18.0500,30.9500,305.60,18,0,9.32,SD,1697143027
64,64,Crane,31.3974,-102.3501,305.47,17,0,10.43,US,1697143039
68,68,Faya-Largeau,17.9257,19.1043,304.42,20,4,6.31,TD,1697143040
72,72,Taunsa,30.7048,70.6505,299.00,19,0,1.23,PK,1697143040
168,168,Reggane,26.7158,0.1714,302.64,19,3,6.94,DZ,1697143050


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = BestWeather

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,Ad Dabbah,18.0500,30.9500,305.60,18,0,9.32,SD,1697143027,
64,64,Crane,31.3974,-102.3501,305.47,17,0,10.43,US,1697143039,
68,68,Faya-Largeau,17.9257,19.1043,304.42,20,4,6.31,TD,1697143040,
72,72,Taunsa,30.7048,70.6505,299.00,19,0,1.23,PK,1697143040,
168,168,Reggane,26.7158,0.1714,302.64,19,3,6.94,DZ,1697143050,
235,235,Gao,16.6362,1.6370,305.14,12,0,4.30,ML,1697143056,
272,272,Tiaret,35.3710,1.3170,290.84,16,0,2.57,DZ,1697143060,
287,287,Muleshoe,34.2265,-102.7238,299.95,17,0,11.25,US,1697143061,
298,298,Al Kharj,24.1550,47.3346,303.47,16,0,2.32,SA,1697143062,
318,318,Bam,29.1060,58.3570,300.58,16,2,3.53,IR,1697143020,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:

# Set parameters to search for a hotel
radius = 100000
params = {
    "radius": radius,
    "categories": "accommodation",
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Ad Dabbah - nearest hotel: Abdo’s Hostel
Crane - nearest hotel: No hotel found
Faya-Largeau - nearest hotel: No hotel found
Taunsa - nearest hotel: No hotel found
Reggane - nearest hotel: قرية النفيس
Gao - nearest hotel: No hotel found
Tiaret - nearest hotel: Hôtel D'orient
Muleshoe - nearest hotel: Heritage House Inn
Al Kharj - nearest hotel: العسجريه
Bam - nearest hotel: هتل جهانگردی بم
Ouallam - nearest hotel: No hotel found
Taoudenni - nearest hotel: Taoudeni
Riverside - nearest hotel: Wyndham Resort Indio
San Luis - nearest hotel: Hotel Dos Venados
Khash - nearest hotel: مهمانسرا مجتمع واحد سیمان
Tindouf - nearest hotel: محمد بوسبي
Villa Dolores - nearest hotel: Plaza Hotel


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,Ad Dabbah,18.0500,30.9500,305.60,18,0,9.32,SD,1697143027,Abdo’s Hostel
64,64,Crane,31.3974,-102.3501,305.47,17,0,10.43,US,1697143039,No hotel found
68,68,Faya-Largeau,17.9257,19.1043,304.42,20,4,6.31,TD,1697143040,No hotel found
72,72,Taunsa,30.7048,70.6505,299.00,19,0,1.23,PK,1697143040,No hotel found
168,168,Reggane,26.7158,0.1714,302.64,19,3,6.94,DZ,1697143050,قرية النفيس
235,235,Gao,16.6362,1.6370,305.14,12,0,4.30,ML,1697143056,No hotel found
272,272,Tiaret,35.3710,1.3170,290.84,16,0,2.57,DZ,1697143060,Hôtel D'orient
287,287,Muleshoe,34.2265,-102.7238,299.95,17,0,11.25,US,1697143061,Heritage House Inn
298,298,Al Kharj,24.1550,47.3346,303.47,16,0,2.32,SA,1697143062,العسجريه
318,318,Bam,29.1060,58.3570,300.58,16,2,3.53,IR,1697143020,هتل جهانگردی بم


In [6]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "radius": radius,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey,
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'{lat},{lng}'
    params["bias"] = f'{lat},{lng}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)

# Display the map
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)